# Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re

# for tokenization
from sklearn.feature_extraction.text import CountVectorizer
# natto-py
import natto
from natto import MeCab

# for sentiment analysis
# oseti
import oseti
# asari
from asari.api import Sonar

import os
# this explicitly tells python where to find the MeCab dictionary
# replace with the path to the libmecab.dll if you get an error running the MeCab() instantiation below
os.environ['MECAB_PATH']="C:/Program Files/MeCab/bin/libmecab.dll"
# we need to force MeCab to read Japanese text in UTF-8
os.environ['MECAB_CHARSET']='utf-8'

In [3]:
# Increase dataframe column width:
pd.set_option('max_colwidth', 400)

### Reading in the data

In [4]:
tohoku_reviews_df = pd.read_csv('../datasets/rakuten_tohoku_reviews_replies.csv')

In [5]:
tohoku_reviews_df.shape

(7340, 14)

In [6]:
tohoku_reviews_df.head()

,review_id,review_time,review_text,hotel_reply_time,hotel_reply_text,hotel_name,prefecture,overall_score,service_score,location_score,room_score,amenities_score,bathroom_score,food_score
0,voteans_21301527,2024-02-18 01:26:29,家族でのんびり過ごせて良かったです。お料理が美味しかったし、景色も美しかったです。温泉は思ったよりも小さかったですが、泉質は結構よかった。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,4,4,4,4,5
1,voteans_21251321,2024-01-29 12:03:50,ロビーからの鶴の舞橋の景観が素晴らしく、雪景色もあって楽しめました。食事も満腹となるほどの量・品数があり、いずれも美味でした。近くに鶴の飼育もなされており、鶴の鳴き合いの声を聴きながらの散策も特別な雰囲気を感じました。、,2024-02-15 18:11:20,この度は当館にご宿泊いただき誠にありがとうございます。ロビーから見える鶴の舞橋と雪化粧した岩木山の姿は、私共も見ていて飽きないほどです。また、お食事の面でもお客様にご満足いただけたようで大変嬉しく思います。丹頂鶴自然公園は1年を通して鶴とのふれあいを楽しめますので、ご機会がございましたらまたのお越しを心よりお待ち申し上げております。,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,5,4,4,4,4
2,voteans_21170325,2023-12-26 15:04:31,先日はお世話になりました。夕食を部屋食への変更、ありがとうございました。お食事はとても味付けが良くて、量も適量で、本当にゆっくりと食事が出来て良かったです。大浴場のお湯も、しっとりとしてかけ流しのお湯は気持ち良かったです。なかなか遠いのでまたチャンスがあれば伺いたいと思います。鶴の舞橋が目の前で、圧巻でした。雪が舞い散る鶴の舞橋も幻想的でとても良かったです。また足を運びたいと思います。ありがとうございました。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,5,5,5,4,5,5
3,voteans_21061209,2023-11-21 16:12:52,大変満足でした。お掃除も行き届いていましたしお料理も美味しかったです。お風呂のお湯も豊富で気持ちよかったです。とにかく鶴の舞橋が目の前で素晴らしかったです。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,5,4,5,4,3,4,4
4,voteans_21031432,2023-11-12 19:16:54,【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰...,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,2,2,5,2,3,2,2


In [7]:
tohoku_reviews_df.isna().sum()

review_id              0
review_time            0
review_text            0
hotel_reply_time    1070
hotel_reply_text    1070
hotel_name             0
prefecture             0
overall_score          0
service_score          0
location_score         0
room_score             0
amenities_score        0
bathroom_score         0
food_score             0
dtype: int64

## Data Cleaning

Even though most of our reviews are in Japanese, there are still some English reviews. 

We can look at a few by filtering based on whether the review contains two strings of at least 4 latin alphabet characters separated by spaces.

In [8]:
possibly_english_reviews_df = tohoku_reviews_df[tohoku_reviews_df['review_text'].str.contains(
    r'[a-z]{,3} [a-z]{4,} [a-z]{4,}([a-z]{,3})? | ([a-z]{,3} )?[a-z]{4,} [a-z]{4,}[a-z]{,3}'
    )].copy()

C:\Users\weich\AppData\Local\Temp\ipykernel_19236\3445674618.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  possibly_english_reviews_df = tohoku_reviews_df[tohoku_reviews_df['review_text'].str.contains(


In [9]:
possibly_english_reviews_df.iloc[:,:3]

,review_id,review_time,review_text
754,voteans_21289816,2024-02-13 16:00:33,spacious room with separate bathroom and washlet toilet. we tried one day of breakfast which is great for \1100@.
769,voteans_20961951,2023-10-22 21:12:48,"I found myself enjoying the Tatami Room a lot. The budget is reasonable, and the staff were quite friendly. They had a Welcome Drink (Apple Juice) when my family and I checked-in at the hotel. You can serve yourself this beverage once. The apple juice was delicious and sweet. Made a good first-impression.和室のお部屋でしたが、入った時に畳の匂いが良かったです。アメニティもあり、お風呂もゆっくりつかれて、よく眠れました。フロントの方も正にスマイルで対応していただきました。"
3384,voteans_21242782,2024-01-25 18:20:37,"A bit old hotel decoration, rooms are near the railway and quiet noisy. We are not sleep well."
3895,voteans_21242788,2024-01-25 18:22:51,"Excellent hotel support guest needs, we will return next turn."
4855,voteans_21242784,2024-01-25 18:21:47,"Very nice stay experience with good service, nice room and dinning quality is excellent."
4878,voteans_21015482,2023-11-07 15:47:16,The hotel room is aged and not very clean. The buffet dinner and breakfast were not good in taste and a bit mess. The room charge was not worth for the value
5982,voteans_21214669,2024-01-12 17:19:33,"Location was very good and convenient. Room was clean and cozy, breakfast was very delicious. But the outdoor hot spring(in hotel) closed when we visited, felt unsatisfied. Shuttle bus refused to pick us up to the gondola terminal and we didn't know how to book the bus. Service could be improved."
6861,voteans_21268725,2024-02-05 16:48:41,Amazing room very accommating staff the meals were incredible very knowledgeable and friendly service
7305,voteans_21214683,2024-01-12 17:24:39,Breakfast was very delicious. Free parking was very convenient for the self driving visitors. Location is good enough.
7306,voteans_21214678,2024-01-12 17:22:16,Very good location and free parking were nice to visitors. Room was clean and comfortable even bigger than I expected. Breakfast was tasty and healthy.


In [10]:
possibly_english_reviews_df.shape

(10, 14)

There's only 10 out of 7330 reviews that have such English text, so we can safely drop them.

### Analysis of the remaining texts

In [11]:
japanese_reviews_df = tohoku_reviews_df[~tohoku_reviews_df['review_text'].str.contains(
    r'[a-z]{,3} [a-z]{4,} [a-z]{4,}([a-z]{,3})? | ([a-z]{,3} )?[a-z]{4,} [a-z]{4,}[a-z]{,3}'
    )].copy()

C:\Users\weich\AppData\Local\Temp\ipykernel_19236\3010851474.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  japanese_reviews_df = tohoku_reviews_df[~tohoku_reviews_df['review_text'].str.contains(


In [12]:
japanese_reviews_df.iloc[:,:3].head()

,review_id,review_time,review_text
0,voteans_21301527,2024-02-18 01:26:29,家族でのんびり過ごせて良かったです。お料理が美味しかったし、景色も美しかったです。温泉は思ったよりも小さかったですが、泉質は結構よかった。
1,voteans_21251321,2024-01-29 12:03:50,ロビーからの鶴の舞橋の景観が素晴らしく、雪景色もあって楽しめました。食事も満腹となるほどの量・品数があり、いずれも美味でした。近くに鶴の飼育もなされており、鶴の鳴き合いの声を聴きながらの散策も特別な雰囲気を感じました。、
2,voteans_21170325,2023-12-26 15:04:31,先日はお世話になりました。夕食を部屋食への変更、ありがとうございました。お食事はとても味付けが良くて、量も適量で、本当にゆっくりと食事が出来て良かったです。大浴場のお湯も、しっとりとしてかけ流しのお湯は気持ち良かったです。なかなか遠いのでまたチャンスがあれば伺いたいと思います。鶴の舞橋が目の前で、圧巻でした。雪が舞い散る鶴の舞橋も幻想的でとても良かったです。また足を運びたいと思います。ありがとうございました。
3,voteans_21061209,2023-11-21 16:12:52,大変満足でした。お掃除も行き届いていましたしお料理も美味しかったです。お風呂のお湯も豊富で気持ちよかったです。とにかく鶴の舞橋が目の前で素晴らしかったです。
4,voteans_21031432,2023-11-12 19:16:54,【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰...


We first check that none of the reviews are blank.

In [13]:
japanese_reviews_df[japanese_reviews_df['review_text']=='']

,review_id,review_time,review_text,hotel_reply_time,hotel_reply_text,hotel_name,prefecture,overall_score,service_score,location_score,room_score,amenities_score,bathroom_score,food_score


In addition, some of the scores given in the reviews may not be complete. Let us take a look.

In [14]:
japanese_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7330 entries, 0 to 7339
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   review_id         7330 non-null   object
 1   review_time       7330 non-null   object
 2   review_text       7330 non-null   object
 3   hotel_reply_time  6260 non-null   object
 4   hotel_reply_text  6260 non-null   object
 5   hotel_name        7330 non-null   object
 6   prefecture        7330 non-null   object
 7   overall_score     7330 non-null   int64 
 8   service_score     7330 non-null   int64 
 9   location_score    7330 non-null   int64 
 10  room_score        7330 non-null   int64 
 11  amenities_score   7330 non-null   object
 12  bathroom_score    7330 non-null   object
 13  food_score        7330 non-null   object
dtypes: int64(4), object(10)
memory usage: 859.0+ KB


The reason why some scores are not in integer format is because when a guest does not have a score for a specific aspect, the review shows it as `-`.

We shall convert them to null values and check how many there are.

In [15]:
# applying the filter to our dataset

japanese_reviews_df.replace('-', np.nan, inplace=True)

In [16]:
japanese_reviews_df.isna().sum()

review_id              0
review_time            0
review_text            0
hotel_reply_time    1070
hotel_reply_text    1070
hotel_name             0
prefecture             0
overall_score          0
service_score          0
location_score         0
room_score             0
amenities_score       25
bathroom_score       321
food_score          1616
dtype: int64

We can leave the null scores be for now, and only remove them later when needed. 

We also see that around 1000 of the 7330 reviews do not have a reply. This is fine.

Meanwhile, let us take a look at the range of scores, just so we get a feel for what we are dealing with.

In [17]:
for col in [colname for colname in japanese_reviews_df.columns.tolist() if 'score' in colname]:
    japanese_reviews_df[col] = [int(score) if score == score else np.nan for score in japanese_reviews_df[col].values]

In [18]:
{col: japanese_reviews_df[col].unique() for col in [colname for colname in japanese_reviews_df.columns if 'score' in colname]}

{'overall_score': array([4, 5, 2, 3, 1], dtype=int64),
 'service_score': array([4, 5, 2, 3, 1], dtype=int64),
 'location_score': array([4, 5, 3, 2, 1], dtype=int64),
 'room_score': array([4, 5, 2, 3, 1], dtype=int64),
 'amenities_score': array([ 4.,  3.,  2.,  5.,  1., nan]),
 'bathroom_score': array([ 4.,  5.,  2.,  3., nan,  1.]),
 'food_score': array([ 5.,  4.,  2.,  3., nan,  1.])}

As we can see, we have the full range of scores for each category, so we can attempt to use these to check the sentiments.

## Tokenization and Aspect Detection

As the goal of this project is Aspect Based Sentiment Analysis, we would like to first identify what the aspects of the reviews are. If we cannot even determine that, we would just be performing Sentiment Analysis.


### Tokenization

In order to tokenize sentences in the Japanese language, we can use the `natto-py` library (documentation [here](https://pypi.org/project/natto-py/)), self-described as "A Tasty Python Binding with MeCab (FFI-based, no SWIG or compiler necessary)".

Note that `natto-py` also requires a [MeCab](https://en.wikipedia.org/wiki/MeCab) installation (essentially the dictionary to be used) to be present in order for the tokenizer to even work. This can be downloaded from https://github.com/ikegami-yukino/mecab/releases (download and run `mecab-64-0.996.2.exe`), and the encoding should be set to UTF-8. It should be noted that the [official documentation for MeCab](https://taku910.github.io/mecab/learn.html) is written pretty much entirely in Japanese, and so we shall explore it through its wrappers, which tend to have documentation in English.

For more information, refer to the [official `natto-py` wiki](https://github.com/buruzaemon/natto-py/wiki) for documentation and examples. The author of `natto-py` describes the motivation behind creating it [here](https://github.com/buruzaemon/natto-py/wiki/Why-Even-Bother).

Alternatively, `fugashi` (available [here](https://github.com/polm/fugashi), which is much easier in terms of a dictionary installation) could also be used. However, the advantage that `natto-py` provides over `fugashi` is not just a much more comprehensive documentation, but it also does not require explicit configuration of MeCab besides making sure it is on your system PATH.

A comparison of the different tokenizers is provided in [this](https://arxiv.org/abs/2010.06858) paper (article version [here](https://www.dampfkraft.com/nlp/how-to-tokenize-japanese.html)), which introduces the `fugashi` library and also provides a general overview of Japanese tokenization. In fact, most Japanese language tokenizers available for Python are simply wrappers for MeCab.

There is another tokenizer that we may want to explore here. [Nagisa](https://github.com/taishi-i/nagisa) is a Japanese language tokenizer that is based off a recurrent neural network. 

### Aspect Detection

In this attempt to programatically determine aspects, let us look at the most common tokens that occur in our corpus of text. We shall use `scikit-learn`'s `CountVectorizer` here, with the only difference being that we pass a different tokenizer into `CountVectorizer`.

First, we define the tokenizer that we shall pass into `CountVectorizer`.

In [19]:
# instantiating the MeCab() object
nm = MeCab()

def init_tokenizer_ja(text):
    # this essentially returns a list of tokens that are not end-of-sentence (EOS)
    return [token.surface for token in nm.parse(text, as_nodes=True) if token.is_nor()]

Let us see this in action.

In [20]:
init_tokenizer_ja('家族でのんびり過ごせて良かったです。')

['家族', 'で', 'のんびり', '過ごせ', 'て', '良かっ', 'た', 'です', '。']

In [21]:
example_text = '先日はお世話になりました。夕食を部屋食への変更、ありがとうございました。お食事はとても味付けが良くて、量も適量で、本当にゆっくりと食事が出来て良かったです。大浴場のお湯も、しっとりとしてかけ流しのお湯は気持ち良かったです。なかなか遠いのでまたチャンスがあれば伺いたいと思います。鶴の舞橋が目の前で、圧巻でした。雪が舞い散る鶴の舞橋も幻想的でとても良かったです。また足を運びたいと思います。ありがとうございました。'


In [22]:
example_tokens = [token for token in nm.parse(example_text, as_nodes=True)]

In [23]:
example_tokens

[<natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50B1E8>, stat=0, surface="先日", feature="名詞,副詞可能,*,*,*,*,先日,センジツ,センジツ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50B6C8>, stat=0, surface="は", feature="助詞,係助詞,*,*,*,*,は,ハ,ワ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50BAD8>, stat=0, surface="お世話", feature="名詞,サ変接続,*,*,*,*,お世話,オセワ,オセワ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50BDB0>, stat=0, surface="に", feature="助詞,格助詞,一般,*,*,*,に,ニ,ニ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50C498>, stat=0, surface="なり", feature="動詞,自立,*,*,五段・ラ行,連用形,なる,ナリ,ナリ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50C6A0>, stat=0, surface="まし", feature="助動詞,*,*,*,特殊・マス,連用形,ます,マシ,マシ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50CBE8>, stat=0, surface="た", feature="助動詞,*,*,*,特殊・タ,基本形,た,タ,タ">,
 <natto.node.MeCabNode node=<cdata 'mecab_node_t *' 0x000001E0CF50CD20

In [24]:
example_tokens[0].feature

'名詞,副詞可能,*,*,*,*,先日,センジツ,センジツ'

In [25]:
example_tokens[0].feature.split(',')

['名詞', '副詞可能', '*', '*', '*', '*', '先日', 'センジツ', 'センジツ']

There are many things that show up here, but the important things to keep in mind are that the element with index 0 of this list is the word type of the token (noun, verb, punctuation, etc.), and the element with index -3 of this list is the root (or lemma) of the token.

We see that we have punctuation and some particles (e.g., で and です) here. Particles are essentially like the connector stop words in English.

The great thing about `natto-py` is that this information is also available in the tokenizer, and so we can extract and filter based on it.

In [26]:
# defining the filters for position of speech and stem of the token

def get_token_data(token: natto.node.MeCabNode, data: str):
    '''Gets the relevant data from `token`

    Parameters
    ---
    - `token`: A MeCab node
    - `data`: A string representing the data to be extracted
        - `'word'`: The word representation of the token
        - `'pos'`: The part of speech that the token belongs to
        - `'lemma'`: The lemma of the token
    '''
    
    # splits token data into a list of features
    token_features = token.feature.split(',')

    if data == 'word':
        return token.surface
    
    if data == 'pos':
        return token_features[0]
    
    if data == 'lemma':
        return token_features[-3]

Now, we can define a new tokenizer that doesn't just take out punctuation (記号) and stop words, but also stems the words.

For the sake of simplicity, we shall treat only particles (助詞), prefixes (接頭詞), conjunctions (接続詞), thanks/greetings (感動詞), and auxillary verbs (助動詞) as stop words (stemming would get rid of the latter either way). 

In [27]:
def tokenizer_ja(text: str):
    '''Returns a list of stemmed tokens that excludes punctuation and grammar.
    '''
    # extracts out the nodes with token data into a list
    token_nodes = [token for token in nm.parse(text, as_nodes=True) if token.is_nor()]

    # removes punctuation and stop particles
    excluded_pos = ['記号',     # punctuation
                    '助詞',     # particles
                    '助動詞',   # auxillary verbs
                    '接頭詞',   # prefixes
                    '接続詞',   # conjunctions
                    '感動詞'    # exclamations of thanks / greetings
                    ]
    token_nodes_no_punc_stop = [token for token in token_nodes if get_token_data(token, 'pos') not in excluded_pos]

    # stems the tokens
    stemmed_tokens = [get_token_data(token, 'lemma') for token in token_nodes_no_punc_stop]

    return stemmed_tokens

def tokenizer_nouns_ja(text: str):
    '''Returns a list of stemmed nouns that excludes punctuation and grammar.
    '''
    # extracts out the nodes with token data into a list
    token_nodes = [token for token in nm.parse(text, as_nodes=True) if token.is_nor()]

    # extracts out only the nouns
    token_nodes_nouns = [token for token in token_nodes if get_token_data(token, 'pos') == '名詞']

    # stems the tokens
    stemmed_tokens = [get_token_data(token, 'lemma') for token in token_nodes_nouns]

    return stemmed_tokens
    
    

Let's see this in action.

In [28]:
tokenizer_ja(example_text)

['先日',
 'お世話',
 'なる',
 '夕食',
 '部屋',
 '食',
 '変更',
 '食事',
 'とても',
 '味付け',
 '良い',
 '量',
 '適量',
 '本当に',
 'ゆっくり',
 '食事',
 '出来る',
 '良い',
 '浴場',
 'お湯',
 'しっとり',
 'する',
 'かける',
 '流す',
 'お湯',
 '気持ち良い',
 'なかなか',
 '遠い',
 'チャンス',
 'ある',
 '伺う',
 '思う',
 '鶴の舞橋',
 '目',
 '前',
 '圧巻',
 '雪',
 '舞う',
 '散る',
 '鶴の舞橋',
 '幻想',
 '的',
 'とても',
 '良い',
 '足',
 '運ぶ',
 '思う']

Works like a charm.

We can now throw this into `CountVectorizer` and look at the words with the highest frequency.

In [29]:
# defining a custom list of Japanese stop words
stop_words_ja = ['する', 'ある', 'いる', 'れる', 'せる', 'なる', 'こと', 'ない', 'もの', '何', '月', '日', '時', 'の']

In [30]:
def word_frequency_plotter(dataframe: pd.DataFrame,
                           n_grams: int = 1,
                           top: int = 30,
                           plot: bool = True,
                           nouns_only: bool = False):
    '''Trains a CountVectorizer instance on the Rakuten hotel reviews.

    Returns
    ---
    A pair `(CountVectorizer, sorted dictionary of tokens and frequencies)`
    - CountVectorizer object, fitted on the reviews in the DataFrame
    - A sorted dictionary
        - Keys are tokens in the vectorizer's dictionary
        - Values are the respective document frequencies as a percentage of the total
    - A plot of the respective top `top` word frequencies

    Parameters
    ---
    - `dataframe`: A pandas DataFrame
    - `n-grams`: int. default=1
        - Number of consecutive tokens to consider
    - `top`: int. default=30
        - Returns a plot of the top `top` words in the corpus.
    - `plot`: bool. default=True
        - If False, returns only the fitted CountVectorizer and dictionary of word frequencies
    - `nouns_only`: bool. default=False
        - If True, returns only the tokenized nouns.
    '''

    # gets reviews
    reviews = dataframe['review_text']

    if nouns_only:
        tokenizer = tokenizer_nouns_ja
    else:
        tokenizer = tokenizer_ja

    # initializes CountVectorizer() with our custom Japanese tokenizer
    cvec_ja = CountVectorizer(tokenizer=tokenizer, 
                              ngram_range=(n_grams, n_grams),
                              stop_words=stop_words_ja)

    # fits the CountVectorizer on the corpus
    cvec_ja_sparse = cvec_ja.fit_transform(reviews)

    # gets array of word counts
    # this uses the .sum() method for numpy sparse matrices
    # it is much more efficient than converting to a dense matrix and summing that 
    cvec_ja_sums = np.asarray(cvec_ja_sparse.sum(axis=0)).ravel()

    # generates dictionary of tokens with total counts
    count_dict = {word: cvec_ja_sums[cvec_ja.vocabulary_[word]] for word in cvec_ja.get_feature_names_out()}

    # sorts the resulting dictionary in decreasing order (higher frequencies first)
    sorted_dict = {key: value for key, value in sorted(count_dict.items(), key=lambda x: x[1], reverse=True)}

    if plot:
        # we need MS Gothic in order to render Japanese text in the plot
        # matplotlib will try sans-serif first
        # if the text cannot be rendered in sans-serif
        # then matplotlib tries MS Gothic
        plt.rcParams['font.family'] = ['sans-serif', 'MS Gothic']

        # plots the frequencies
        plt.figure(figsize=(20,10))
        plt.barh(y=list(sorted_dict.keys())[:top][::-1], width=list(sorted_dict.values())[:top][::-1])
        plt.title(f'Frequency of top {top} most common {n_grams}-gram{" nouns" if nouns_only else "s"}')
        plt.xlabel(f'Number of occurrences of the given {"noun" if nouns_only else "word"}')
        plt.ylabel(f'Most common {"nouns" if nouns_only else "words"}')

    return cvec_ja, dict(list(sorted_dict.items())[:top])

In [31]:
word_frequency_plotter(japanese_reviews_df, n_grams=1, top=100, plot=False, nouns_only=True)

c:\Users\weich\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(CountVectorizer(stop_words=['する', 'ある', 'いる', 'れる', 'せる', 'なる', 'こと', 'ない',
                             'もの', '何', '月', '日', '時', 'の'],
                 tokenizer=<function tokenizer_nouns_ja at 0x000001E0D85F2A20>),
 {'部屋': 4123,
  '利用': 2539,
  'ホテル': 2030,
  '朝食': 1996,
  '宿泊': 1935,
  '食事': 1708,
  '風呂': 1702,
  '満足': 1481,
  '温泉': 1329,
  '方': 1277,
  'スタッフ': 1024,
  '対応': 1002,
  '駅': 998,
  '人': 899,
  '残念': 895,
  '浴場': 836,
  '大変': 813,
  '場': 787,
  'サービス': 756,
  '最高': 741,
  '立地': 723,
  'よう': 717,
  '料理': 712,
  '時間': 711,
  'フロント': 708,
  '便利': 678,
  '綺麗': 677,
  '的': 674,
  'さ': 669,
  '夕食': 648,
  '近く': 647,
  '快適': 637,
  '駐車': 637,
  'さん': 634,
  '宿': 616,
  '今回': 608,
  '気': 587,
  'バイキング': 550,
  '朝': 541,
  '予約': 514,
  '泊': 509,
  '事': 507,
  '中': 506,
  'チェックイン': 501,
  '前': 492,
  '露天風呂': 475,
  'ため': 474,
  'バス': 460,
  '感じ': 450,
  '一': 448,
  '種類': 418,
  '清潔': 403,
  '感': 400,
  '私': 400,
  '丁寧': 397,
  'ところ': 392,
  '際': 387,
  'アメニティ': 373,
  'コンビニ': 37

### Bringing it back to reviews

Recall that we have 6 different aspects to consider - Service, Location, Room, Amenities, Bathroom, Food.

We shall use the keywords obtained by a brief scan of the most commonly used nouns and adjectives in the corpus to identify the different aspects of a review, if any.

The keywords identified so far are as follows:

|Aspect|Definition|Keywords|
|---|---|---|
|Service|Acts of help towards customer satisfaction|サービス, スタッフ, フロント, チェックイン, 丁寧, 親切, 接客, サーバ|
|Location|Access and landscape around the hotel|立地, 駅, バス, 近く, 便利, 駐車, コンビニ, 場所|
|Room|The attributes of the room|部屋, 広い, 宿泊, ベッド, 値段|
|Amenities|Other facilities in the hotel excluding the room and bath|アメニティ, 無料|
|Bathroom|Bath in the hotel room, or a public bath|風呂, 温泉, 浴場, 露天風呂, 清潔, 湯, トイレ|
|Food|Morning or evening meals|朝食, 食事, 料理, 夕食, バイキング, メニュー, ご飯,  酒, 飲|

Let us now try to identify different aspects in an example review, taken from one of our data points.

In [32]:
example_review_for_aspects = tohoku_reviews_df['review_text'][4]
example_review_for_aspects

'【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）。地元の方にも人気のある温泉のようで夕方は混み合う。そして顔見知りの方々の会話が長く続いている。マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）。また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない。改善されれば幸いです。'

Given a review, we would want to break it up into tokens (this time sentence-based) and classify them based on the keywords present. Let us first define the different categories as per the table above.

In [33]:
# aspect categories
aspect_words = {
    'service': ['サービス', 'スタッフ', 'フロント', 'チェックイン', '丁寧', '親切', '接客', 'サーバ'],
    'location': ['立地', '駅', 'バス', '近く', '便利', '駐車', 'コンビニ', '場所'],
    'room': ['部屋', '広い', '宿泊', 'ベッド', '値段'], 
    'amenities': ['アメニティ', '無料'],
    'bathroom': ['風呂', '温泉', '浴場', '露天風呂', '清潔', '湯', 'トイレ'],
    'food': ['朝食', '食事', '料理', '夕食', 'バイキング', 'メニュー', 'ご飯', '酒', '飲']
}

words_aspect = {word: aspect for aspect, word_list in aspect_words.items() for word in word_list}
words_aspect

{'サービス': 'service',
 'スタッフ': 'service',
 'フロント': 'service',
 'チェックイン': 'service',
 '丁寧': 'service',
 '親切': 'service',
 '接客': 'service',
 'サーバ': 'service',
 '立地': 'location',
 '駅': 'location',
 'バス': 'location',
 '近く': 'location',
 '便利': 'location',
 '駐車': 'location',
 'コンビニ': 'location',
 '場所': 'location',
 '部屋': 'room',
 '広い': 'room',
 '宿泊': 'room',
 'ベッド': 'room',
 '値段': 'room',
 'アメニティ': 'amenities',
 '無料': 'amenities',
 '風呂': 'bathroom',
 '温泉': 'bathroom',
 '浴場': 'bathroom',
 '露天風呂': 'bathroom',
 '清潔': 'bathroom',
 '湯': 'bathroom',
 'トイレ': 'bathroom',
 '朝食': 'food',
 '食事': 'food',
 '料理': 'food',
 '夕食': 'food',
 'バイキング': 'food',
 'メニュー': 'food',
 'ご飯': 'food',
 '酒': 'food',
 '飲': 'food'}

Now we take a review and break it up into sentence-like fragments (separated by periods). For simplicity, we shall first assume that each aspect is contained within a sentence (if any). For aspects which span at least 2 sentences, we would leave them be for now.

In [34]:
def has_aspect(aspect: str, text: str):
    '''Returns a boolean describing whether a document contains an aspect word or not
    '''
    # this returns True if it exists in the list
    # otherwise returns False
    return max([(word in text) for word in aspect_words[aspect]])

In [35]:
# test cases
print(has_aspect('service', example_review_for_aspects))
print(has_aspect('amenities', example_review_for_aspects))

True
False


In [36]:
# extracting existing aspects in a review
example_review_for_aspects_aspects = [aspect for aspect in aspect_words if has_aspect(aspect, example_review_for_aspects)]
example_review_for_aspects_aspects

['service', 'room', 'bathroom', 'food']

In [37]:
# assigning aspects to each sentence in the review that has any
def label_sentence_aspects(text):
    '''Returns a dictionary of sentences in `text` with their corresponding aspects.
    '''
    list_of_aspects = [aspect for aspect in aspect_words if has_aspect(aspect, text)]
    
    aspect_decomposition = {}

    for sentence in re.split('。', text):
        for aspect in list_of_aspects:
            if sentence not in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[sentence] = [aspect]
            elif sentence in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[sentence].append(aspect)
    
    return aspect_decomposition



From here, we can attempt to assign a tag for the aspect that each sentence has.

In [38]:
label_sentence_aspects(example_review_for_aspects)

{'【良かった点】・部屋から見える景色が素晴らしく大満足': ['room'],
 '・フロントの対応が丁寧': ['service'],
 '【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと': ['service',
  'room'],
 '・部屋の髪の毛白いソファーに長い髪の毛が2本': ['room'],
 '・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた': ['service',
  'room',
  'food'],
 '食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった': ['food'],
 '飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）': ['food'],
 '朝食は席に着いても説明がなく戸惑った': ['food'],
 '・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）': ['bathroom'],
 '地元の方にも人気のある温泉のようで夕方は混み合う': ['bathroom'],
 'また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない': ['room', 'food']}

In fact, let us change the output of the sentence tagger a tad bit, just to make sentiment analysis easier.

In [39]:
# assigning aspects to each sentence in the review that has any
def label_aspects_sentence(text):
    '''Returns a dictionary of aspects in `text` with their corresponding sentences.
    '''
    list_of_aspects = [aspect for aspect in aspect_words if has_aspect(aspect, text)]
    
    aspect_decomposition = {}

    for sentence in re.split('。', text):
        for aspect in list_of_aspects:
            # all we did here is to swap the aspects and the sentences in the key:value pairs
            if aspect not in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[aspect] = [sentence]
            elif aspect in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[aspect].append(sentence)
    
    return aspect_decomposition



In [40]:
label_aspects_sentence(example_review_for_aspects)

{'room': ['【良かった点】・部屋から見える景色が素晴らしく大満足',
  '【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと',
  '・部屋の髪の毛白いソファーに長い髪の毛が2本',
  '・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた',
  'また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない'],
 'service': ['・フロントの対応が丁寧',
  '【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと',
  '・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた'],
 'food': ['・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた',
  '食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった',
  '飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）',
  '朝食は席に着いても説明がなく戸惑った',
  'また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない'],
 'bathroom': ['・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）',
  '地元の方にも人気のある温泉のようで夕方は混み合う']}

Works like a charm. We will now keep this for the sentiment analyzer to pick up later.

This now allows us to filter for parts of the review by aspects, and also has the advantage of ignoring sentences which do not contain any aspect.

However, a possible drawback to this sentence-based approach is that aspect chunks that span multiple sentences will not get picked up.

# Sentiment Analysis in Japanese

While quite a number of sentiment analyzers in python are available for the Japanese language, we first use the [`oseti` library](https://github.com/ikegami-yukino/oseti) for this project. The word `oseti` references the Japanese "osechi ryouri", a term used to refer to a feast eaten during the Japanese New Year.

The reason is that many Japanese sentiment analyzers available are pre-trained models using BERT, such as [a model trained on sentences from Amazon reviews](https://huggingface.co/christian-phu/bert-finetuned-japanese-sentiment), and [a model trained on the chABSA dataset](https://github.com/jarvisx17/Sentiment-Analysis), which is a (rather well-known) publicly available dataset of Japanese financial reports labeled by aspect-based sentiment. 

Another publicly available model that many of the BERT models is based on would be the `BERT-base` and `BERT-large` models trained on Japanese Wikipedia by researchers from the [Tohoku NLP Group](https://github.com/cl-tohoku/bert-japanese/tree/v2.0). Though they all work well for their respective contexts, a concern is that they would not be able to fit our context too well.

On the other hand, `oseti` works purely using a dictionary-based approach, so that it may perhaps fit better to our context of hotel review analysis.

That said, if time permits, we may try out the other models and see how they fare against `oseti`.

In [41]:
# initializing the analyzer
init_analyzer = oseti.Analyzer()

Let's try it out.

In [42]:
broken_up_example_review = re.split('。', example_review_for_aspects)
broken_up_example_review

['【良かった点】・部屋から見える景色が素晴らしく大満足',
 '・フロントの対応が丁寧',
 '【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと',
 '施設外まで出る必要があり、イマイチのんびり出来なかった',
 '・部屋の髪の毛白いソファーに長い髪の毛が2本',
 'ソファーが白なので目立ちました',
 '・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた',
 '食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった',
 '飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）',
 '朝食は席に着いても説明がなく戸惑った',
 '・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）',
 '地元の方にも人気のある温泉のようで夕方は混み合う',
 'そして顔見知りの方々の会話が長く続いている',
 'マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）',
 'また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない',
 '改善されれば幸いです',
 '']

In [43]:
init_analyzer.analyze(broken_up_example_review[0])

[1.0]

In [44]:
init_analyzer.analyze_detail(broken_up_example_review[0])

[{'positive': ['良い', '景色', '素晴らしい', '満足'], 'negative': [], 'score': 1.0}]

In [45]:
init_analyzer.analyze(example_review_for_aspects)

[1.0,
 1.0,
 0.2,
 0.0,
 0,
 1.0,
 -0.6666666666666666,
 0,
 0.0,
 -1.0,
 0.0,
 0.3333333333333333,
 0,
 0,
 0,
 1.0]

In [46]:
list(zip(broken_up_example_review, init_analyzer.analyze_detail(example_review_for_aspects)))

[('【良かった点】・部屋から見える景色が素晴らしく大満足',
  {'positive': ['良い', '景色', '素晴らしい', '満足'], 'negative': [], 'score': 1.0}),
 ('・フロントの対応が丁寧', {'positive': ['丁寧'], 'negative': [], 'score': 1.0}),
 ('【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと',
  {'positive': ['不足-NEGATION', '喫煙-NEGATION', '禁煙'],
   'negative': ['禁煙-NEGATION', '禁煙-NEGATION'],
   'score': 0.2}),
 ('施設外まで出る必要があり、イマイチのんびり出来なかった',
  {'positive': ['出来る'], 'negative': ['イマイチ'], 'score': 0.0}),
 ('・部屋の髪の毛白いソファーに長い髪の毛が2本', {'positive': [], 'negative': [], 'score': 0.0}),
 ('ソファーが白なので目立ちました', {'positive': ['目立つ'], 'negative': [], 'score': 1.0}),
 ('・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた',
  {'positive': ['料金-NEGATION'],
   'negative': ['質-NEGATION',
    'サービス-NEGATION',
    '質-NEGATION',
    '量-NEGATION',
    '質-NEGATION'],
   'score': -0.6666666666666666}),
 ('食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった',
  {'positive': [], 'negative': [], 'score': 0.0}),
 ('飲み物を追加したいが、係の人が来るまで5分程度待った（大声

We see that `oseti` analyzes the tokens based on whether they are marked as positive or not. This sentiment analyzer in particular takes into account negation and the double negatives so often seen in the Japanese language.

However, this is not perfect. As `oseti` is purely dictionary-based, it can miss out some contextual clues that may change the meaning of the sentence. 

For instance, the last sentence "改善されれば幸いです" ("kaizen sarereba saiwai desu"), which translates to "It would be good if this could be improved" is in fact a negative statement expressing displeasure, but incorrectly tagged as positive because the words 改善 (improvement) and 幸い (for the best) are tagged individually as positive words.

Thus, let us look at a more powerful sentiment analyzer called `asari`, which is able to pick up on more nuanced sentiments.

### Asari

Behold, the power of `asari`.

In [47]:
# initializing asari sentiment analyzer
sonar = Sonar()

In [48]:
sonar.ping(example_review_for_aspects)

{'text': '【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）。地元の方にも人気のある温泉のようで夕方は混み合う。そして顔見知りの方々の会話が長く続いている。マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）。また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない。改善されれば幸いです。',
 'top_class': 'negative',
 'classes': [{'class_name': 'negative', 'confidence': 0.5721551775932312},
  {'class_name': 'positive', 'confidence': 0.4278448224067688}]}

In [49]:
[sonar.ping(sentence) for sentence in broken_up_example_review[:-1]]

[{'text': '【良かった点】・部屋から見える景色が素晴らしく大満足',
  'top_class': 'positive',
  'classes': [{'class_name': 'negative', 'confidence': 0.018676722422242165},
   {'class_name': 'positive', 'confidence': 0.9813232421875}]},
 {'text': '・フロントの対応が丁寧',
  'top_class': 'positive',
  'classes': [{'class_name': 'negative', 'confidence': 0.294780969619751},
   {'class_name': 'positive', 'confidence': 0.705219030380249}]},
 {'text': '【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと',
  'top_class': 'positive',
  'classes': [{'class_name': 'negative', 'confidence': 0.4330638349056244},
   {'class_name': 'positive', 'confidence': 0.5669361352920532}]},
 {'text': '施設外まで出る必要があり、イマイチのんびり出来なかった',
  'top_class': 'negative',
  'classes': [{'class_name': 'negative', 'confidence': 0.5682018995285034},
   {'class_name': 'positive', 'confidence': 0.43179813027381897}]},
 {'text': '・部屋の髪の毛白いソファーに長い髪の毛が2本',
  'top_class': 'negative',
  'classes': [{'class_name': 'negative', 'co

Again, context tells us that `asari` isn't perfect either, though it certainly performs much better than `oseti`. 

For instance, the sentence "マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）", which translates to "Regarding talking without masks on, the hotel's side did not have any special instructions (not even posters, etc.)"

In the context of the recent COVID-19 pandemic, this would be interpreted as a negative review from a health-conscious guest. In fact, this particular guest actually organized their review in such a way that the positive feedback is all grouped under a "【良かった点】" tag (which means "good points"), and all the negative feedback is grouped under a "【気になった点】" tag (which means something along the lines of "worrying points").

The majority of the review is under the latter tag, which does indeed match with most of the text except for the sentences regarding the lack of mask-wearing.

# Feature Engineering - Aspect-Based Sentiment Scores

We now shift our attention to identifying the aspects present in each review, and do the following:
- Compare the overall sentiment per aspect to the actual score given (if any)
- Extract the frequencies of the most common nouns that come up with each aspect.

As a reminder, we currently have the function `label_aspect_sentence()` from above, which takes in an individual review and outputs a dictionary of the aspects present in the review, and the sentences that fall under that aspect.

We shall use this and the `asari` sentiment analyzer to compute an averaged sentiment score for each review. It is an extremely naïve way to perform an aspect-based sentiment analysis, but in the interest of time, let us see how far this can take us.

In [50]:
def compute_aspect_sentiment_score_asari(row, analyzer: Sonar, aspect: str):
    '''Uses the `asari` sentiment analyzer to assign each review an 
    averaged sentiment score from all the sentences with a specific aspect `aspect`. 

    This requires an already instantiated Sonar object from `asari`.

    To be used with `df.apply()`.
    '''

    review_text = row['review_text']

    # extracts all the aspects present in the review
    # together with their corresponding sentences in the review
    aspect_sectence_dict = label_aspects_sentence(review_text)

    ##############################################################################
    # helper function to extract positive and negative sentiment confidence scores
    def get_pos_neg_sentiments(text):
        '''This is a helper function to extract out the positive and negative sentiment confidence levels in `text`.

        Returns
        ---
        A tuple `(pos, neg)`, where `pos` is the positive sentiment confidence, and `neg` is the negative sentiment confidence.
        '''
        sentiment_data = analyzer.ping(text)
            
        # sentiments are split into positive and negative only
        # the output of Sonar().ping is a dictionary, 
        # where the 'classes' key contains the confidence levels pos and neg of each class
        # this is such that pos + neg = 1, think of it as a Bernoulli probability distribution
        pos = sentiment_data['classes'][1]['confidence']
        neg = sentiment_data['classes'][0]['confidence']

        return pos, neg
    ##############################################################################

    if aspect == 'all':
        pos, neg = get_pos_neg_sentiments(review_text)
        # the score we shall assign each sentence is pos - neg, essentially the net sentiment
        net_sentiment = pos - neg
        return net_sentiment
    
    # checks if the review contains the given aspect
    elif aspect in aspect_sectence_dict:
        sentiments = []
        for sentence in aspect_sectence_dict[aspect]:
            pos, neg = get_pos_neg_sentiments(sentence)

            # the score we shall assign each sentence is pos - neg, essentially the net sentiment
            net_sentiment = pos - neg
            sentiments.append(net_sentiment)
        
        return np.mean(sentiments)
    
    else: # if aspect is not present in review
        return 0


Now, we populate the sentiment scores for each aspect.

In [51]:
# helper lookup table for easier population
aspect_lookup_table = {
    'all': 'overall',
    'service': 'service',
    'location': 'location',
    'room': 'room',
    'amenities': 'amenities',
    'bathroom': 'bathroom',
    'food': 'food'
}

The sentiment scores will take around 5 minutes to populate. Please be patient when running this.

In [52]:
for aspect in aspect_lookup_table:
    japanese_reviews_df[f'{aspect_lookup_table[aspect]}_sentiment_asari'] = japanese_reviews_df.apply(compute_aspect_sentiment_score_asari, 
                                                                                                      axis=1,
                                                                                                      analyzer=sonar,
                                                                                                      aspect=aspect)
japanese_reviews_df.head()

,review_id,review_time,review_text,hotel_reply_time,hotel_reply_text,hotel_name,prefecture,overall_score,service_score,location_score,...,amenities_score,bathroom_score,food_score,overall_sentiment_asari,service_sentiment_asari,location_sentiment_asari,room_sentiment_asari,amenities_sentiment_asari,bathroom_sentiment_asari,food_sentiment_asari
0,voteans_21301527,2024-02-18 01:26:29,家族でのんびり過ごせて良かったです。お料理が美味しかったし、景色も美しかったです。温泉は思ったよりも小さかったですが、泉質は結構よかった。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,4,...,4.0,4.0,5.0,0.990558,0.000000,0.00000,0.000000,0.0,0.967200,0.887214
1,voteans_21251321,2024-01-29 12:03:50,ロビーからの鶴の舞橋の景観が素晴らしく、雪景色もあって楽しめました。食事も満腹となるほどの量・品数があり、いずれも美味でした。近くに鶴の飼育もなされており、鶴の鳴き合いの声を聴きながらの散策も特別な雰囲気を感じました。、,2024-02-15 18:11:20,この度は当館にご宿泊いただき誠にありがとうございます。ロビーから見える鶴の舞橋と雪化粧した岩木山の姿は、私共も見ていて飽きないほどです。また、お食事の面でもお客様にご満足いただけたようで大変嬉しく思います。丹頂鶴自然公園は1年を通して鶴とのふれあいを楽しめますので、ご機会がございましたらまたのお越しを心よりお待ち申し上げております。,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,5,...,4.0,4.0,4.0,0.980821,0.000000,0.95117,0.000000,0.0,0.000000,0.669060
2,voteans_21170325,2023-12-26 15:04:31,先日はお世話になりました。夕食を部屋食への変更、ありがとうございました。お食事はとても味付けが良くて、量も適量で、本当にゆっくりと食事が出来て良かったです。大浴場のお湯も、しっとりとしてかけ流しのお湯は気持ち良かったです。なかなか遠いのでまたチャンスがあれば伺いたいと思います。鶴の舞橋が目の前で、圧巻でした。雪が舞い散る鶴の舞橋も幻想的でとても良かったです。また足を運びたいと思います。ありがとうございました。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,5,5,...,4.0,5.0,5.0,0.999869,0.000000,0.00000,0.996141,0.0,0.615004,0.977256
3,voteans_21061209,2023-11-21 16:12:52,大変満足でした。お掃除も行き届いていましたしお料理も美味しかったです。お風呂のお湯も豊富で気持ちよかったです。とにかく鶴の舞橋が目の前で素晴らしかったです。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,5,4,5,...,3.0,4.0,4.0,0.988815,0.000000,0.00000,0.000000,0.0,0.909405,0.859946
4,voteans_21031432,2023-11-12 19:16:54,【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰...,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,2,2,5,...,3.0,2.0,2.0,-0.144310,0.325737,0.00000,0.331193,0.0,0.434869,0.079664


For completeness' sake, we can also perform a similar procedure for oseti.

In [53]:
def compute_aspect_sentiment_score_oseti(row, analyzer: oseti.Analyzer, aspect: str):
    '''Uses the `oseti` sentiment analyzer to assign each review an 
    averaged sentiment score from all the sentences with a specific aspect `aspect`. 

    This requires an already instantiated Analyzer object from `oseti`.

    To be used with `df.apply()`.
    '''

    review_text = row['review_text']

    # extracts all the aspects present in the review
    # together with their corresponding sentences in the review
    aspect_sectence_dict = label_aspects_sentence(review_text)

    ##############################################################################
    # helper function to extract positive and negative sentiment confidence scores
    def get_net_sentiments(text):
        '''This is a helper function to extract out the dictionary-based sentiment levels in `text`.

        Returns
        ---
        A net sentiment confidence.
        '''
        sentiment_data = analyzer.analyze(text)

        return np.mean(sentiment_data)
    ##############################################################################

    if aspect == 'all':
        net_sentiment = get_net_sentiments(review_text)
        return net_sentiment
    
    # checks if the review contains the given aspect
    elif aspect in aspect_sectence_dict:
        sentiments = []
        for sentence in aspect_sectence_dict[aspect]:
            net_sentiment = get_net_sentiments(sentence)
            sentiments.append(net_sentiment)
        
        return np.mean(sentiments)
    
    else: # if aspect is not present in review
        return 0


In [54]:
for aspect in aspect_lookup_table:
    japanese_reviews_df[f'{aspect_lookup_table[aspect]}_sentiment_oseti'] = japanese_reviews_df.apply(compute_aspect_sentiment_score_oseti, 
                                                                                                      axis=1,
                                                                                                      analyzer=init_analyzer,
                                                                                                      aspect=aspect)
japanese_reviews_df.head()

,review_id,review_time,review_text,hotel_reply_time,hotel_reply_text,hotel_name,prefecture,overall_score,service_score,location_score,...,amenities_sentiment_asari,bathroom_sentiment_asari,food_sentiment_asari,overall_sentiment_oseti,service_sentiment_oseti,location_sentiment_oseti,room_sentiment_oseti,amenities_sentiment_oseti,bathroom_sentiment_oseti,food_sentiment_oseti
0,voteans_21301527,2024-02-18 01:26:29,家族でのんびり過ごせて良かったです。お料理が美味しかったし、景色も美しかったです。温泉は思ったよりも小さかったですが、泉質は結構よかった。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,4,...,0.0,0.967200,0.887214,1.000000,0.000000,0.0,0.000000,0.0,1.000000,1.000000
1,voteans_21251321,2024-01-29 12:03:50,ロビーからの鶴の舞橋の景観が素晴らしく、雪景色もあって楽しめました。食事も満腹となるほどの量・品数があり、いずれも美味でした。近くに鶴の飼育もなされており、鶴の鳴き合いの声を聴きながらの散策も特別な雰囲気を感じました。、,2024-02-15 18:11:20,この度は当館にご宿泊いただき誠にありがとうございます。ロビーから見える鶴の舞橋と雪化粧した岩木山の姿は、私共も見ていて飽きないほどです。また、お食事の面でもお客様にご満足いただけたようで大変嬉しく思います。丹頂鶴自然公園は1年を通して鶴とのふれあいを楽しめますので、ご機会がございましたらまたのお越しを心よりお待ち申し上げております。,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,4,5,...,0.0,0.000000,0.669060,0.750000,0.000000,1.0,0.000000,0.0,0.000000,1.000000
2,voteans_21170325,2023-12-26 15:04:31,先日はお世話になりました。夕食を部屋食への変更、ありがとうございました。お食事はとても味付けが良くて、量も適量で、本当にゆっくりと食事が出来て良かったです。大浴場のお湯も、しっとりとしてかけ流しのお湯は気持ち良かったです。なかなか遠いのでまたチャンスがあれば伺いたいと思います。鶴の舞橋が目の前で、圧巻でした。雪が舞い散る鶴の舞橋も幻想的でとても良かったです。また足を運びたいと思います。ありがとうございました。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,4,5,5,...,0.0,0.615004,0.977256,0.333333,0.000000,0.0,0.000000,0.0,1.000000,0.500000
3,voteans_21061209,2023-11-21 16:12:52,大変満足でした。お掃除も行き届いていましたしお料理も美味しかったです。お風呂のお湯も豊富で気持ちよかったです。とにかく鶴の舞橋が目の前で素晴らしかったです。,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,5,4,5,...,0.0,0.909405,0.859946,0.750000,0.000000,0.0,0.000000,0.0,1.000000,1.000000
4,voteans_21031432,2023-11-12 19:16:54,【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰...,NaN,NaN,鶴の舞橋と岩木山 絶景の宿 つがる富士見荘,aomori,2,2,5,...,0.0,0.434869,0.079664,0.179167,0.177778,0.0,0.106667,0.0,0.166667,-0.333333


Perfect, now we can export this to another `.csv` file, and start a proper analysis of the sentiments from there.

In [55]:
japanese_reviews_df.to_csv('../datasets/rakuten_tohoku_reviews_sentiments.csv', index=False)